# Notebook 12 (Industrial Edition): Parallel Hybrid Search Fusion for RAG

## Introduction: The Best of Both Worlds for High-Fidelity Retrieval

This notebook explores an advanced RAG pattern designed to maximize retrieval **accuracy and precision**: **Parallel Hybrid Search Fusion**. This technique addresses the fact that no single search algorithm is perfect. Vector search excels at capturing semantic meaning, while traditional keyword search is unbeatable for finding exact matches. A hybrid system runs both in parallel and fuses their results, creating a more robust and comprehensive retrieval engine.

### The Core Concept: Fusing Lexical and Semantic Search

1.  **Vector Search (Semantic):** Finds documents that are contextually related to the query, even if they don't share any keywords.
2.  **Keyword Search (Lexical):** Finds documents that contain the exact terms from the query (e.g., product codes, specific names, acronyms).

By running both searches simultaneously and then combining their unique findings, we create a context that is both semantically relevant and factually precise. This fusion step is critical for preventing failures where one search method misses a crucial document that the other would have found.

### Role in a Large-Scale System: Fusing Diverse Knowledge Types for High-Fidelity Context

In any real-world knowledge base, information is a mix of prose, tables, and identifiers. A hybrid search system is essential for reliably accessing all of it:
- **E-commerce Search:** Finding products by a descriptive query ("warm winter coat") and by a specific model number ("XJ-2000").
- **Legal & Medical Research:** Retrieving documents based on a legal concept (semantic) and those that mention a specific case number or drug name (lexical).
- **Technical Support:** Finding solutions based on a problem description and those that reference a specific error code.

We will build and compare three RAG systems—Vector-Only, Keyword-Only, and Hybrid—to demonstrate how the hybrid approach retrieves a superior context and generates a more **complete and accurate** final answer.

## Part 1: Setup and Environment

We'll need our standard libraries plus `scikit-learn` for its TF-IDF implementation, which we'll use for our keyword search.

In [ ]:
%pip install -U langchain langgraph langsmith langchain-huggingface transformers accelerate bitsandbytes torch langchain-community sentence-transformers faiss-cpu scikit-learn

### 1.2: API Keys and Environment Configuration

We will need our LangSmith and Hugging Face keys.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")
_set_env("HUGGING_FACE_HUB_TOKEN")

# Configure LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Industrial - RAG Hybrid Search"

## Part 2: Components for the Hybrid Search System

We'll need an LLM and two distinct retrieval mechanisms built on the same knowledge base.

### 2.1: The Language Model (LLM)

We will use `meta-llama/Meta-Llama-3-8B-Instruct` as our generator.

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", load_in_4bit=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=False)
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM Initialized. Ready to power our RAG system.")

LLM Initialized. Ready to power our RAG system.


### 2.2: Creating the Knowledge Base & Retrievers

We will create a small knowledge base and then build two different retrievers on top of it.

In [ ]:
from langchain_core.documents import Document

kb_docs = [
    Document(page_content="The QLeap-V4 processor is our latest flagship AI accelerator. It excels at large-scale parallel computations for deep learning workloads.", metadata={"source": "doc-1"}),
    Document(page_content="Project 'Titan' is a new initiative focused on developing energy-efficient hardware. The primary goal is to reduce the power consumption of our data centers.", metadata={"source": "doc-2"}),
    Document(page_content="A key feature of the QLeap-V4 is its advanced thermal management system. The official error code for overheating is 'ERR_THROTTLE_900'.", metadata={"source": "doc-3"}),
    Document(page_content="Our company is committed to sustainability. Project 'Titan' is a core part of our green computing strategy.", metadata={"source": "doc-4"})
]

#### 2.2.1: The Vector Retriever (Semantic)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(kb_docs, embedding=embeddings)
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 2})

#### 2.2.2: The Keyword Retriever (Lexical)

We will use `TfidfVectorizer` from `scikit-learn` to create a classic keyword search index.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List

class TfidfRetriever(BaseRetriever):
    vectorizer: TfidfVectorizer
    docs: List[Document]
    k: int = 2

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        query_vec = self.vectorizer.transform([query])
        doc_vectors = self.vectorizer.transform([doc.page_content for doc in self.docs])
        similarities = cosine_similarity(query_vec, doc_vectors).flatten()
        # Get top_k indices
        top_k_indices = np.argsort(similarities)[-self.k:][::-1]
        return [self.docs[i] for i in top_k_indices]

vectorizer = TfidfVectorizer().fit([doc.page_content for doc in kb_docs])
keyword_retriever = TfidfRetriever(vectorizer=vectorizer, docs=kb_docs, k=2)

## Part 3: Building the RAG Systems

We will build three RAG chains: two baselines and our advanced hybrid system.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

generator_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert technical support agent. Answer the user's question based *only* on the provided context. If the context is insufficient, state that clearly.\n\nContext:\n{context}"),
    ("human", "Question: {question}")
])

def format_docs(docs):
    return "\n\n".join(f"[Source: {doc.metadata.get('source', 'N/A')}] {doc.page_content}" for doc in docs)

# --- RAG Chains ---
rag_chain_vector = ({"context": vector_retriever | format_docs, "question": RunnablePassthrough()} | generator_prompt | llm | StrOutputParser())
rag_chain_keyword = ({"context": keyword_retriever | format_docs, "question": RunnablePassthrough()} | generator_prompt | llm | StrOutputParser())


### 3.1: The Hybrid RAG System Graph

This graph will run our two retrievers in parallel and then fuse their results before generation.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

class HybridRAGState(TypedDict):
    question: str
    retrieved_docs: List[Document]
    final_answer: str

def parallel_retrieval_node(state: HybridRAGState):
    """Runs vector and keyword search in parallel."""
    print("--- [Hybrid Retriever] Running Vector and Keyword searches in parallel... ---")
    vector_docs = vector_retriever.invoke(state['question'])
    keyword_docs = keyword_retriever.invoke(state['question'])
    
    # Fusion: Combine and deduplicate
    all_docs = vector_docs + keyword_docs
    unique_docs = list({doc.page_content: doc for doc in all_docs}.values())
    print(f"--- [Hybrid Retriever] Fused results: Found {len(unique_docs)} unique documents. ---")
    return {"retrieved_docs": unique_docs}

def generation_node(state: HybridRAGState):
    """Generates the final answer."""
    print("--- [Generator] Synthesizing final answer from fused context... ---")
    context = format_docs(state['retrieved_docs'])
    answer = (generator_prompt | llm | StrOutputParser()).invoke({"context": context, "question": state['question']})
    return {"final_answer": answer}

workflow = StateGraph(HybridRAGState)
workflow.add_node("parallel_retrieval", parallel_retrieval_node)
workflow.add_node("generate_answer", generation_node)

workflow.set_entry_point("parallel_retrieval")
workflow.add_edge("parallel_retrieval", "generate_answer")
workflow.add_edge("generate_answer", END)

hybrid_rag_app = workflow.compile()
print("Hybrid RAG graph compiled successfully.")

Hybrid RAG graph compiled successfully.


## Part 4: Head-to-Head Comparison

Let's craft a query designed to fail for single-search systems. It will use a semantic concept ("power saving efforts") and a specific, exact keyword (`ERR_THROTTLE_900`). A good answer requires documents found by both search types.

In [ ]:
user_query = "What are our company's power saving efforts, and what is the error code for QLeap-V4 overheating?"

### 4.1: Running the Vector-Only RAG System

In [ ]:
vector_docs = vector_retriever.invoke(user_query)
vector_answer = rag_chain_vector.invoke(user_query)

print("="*60)
print("                VECTOR-ONLY RAG SYSTEM OUTPUT")
print("="*60 + "\n")
print("Retrieved Documents:")
for i, doc in enumerate(vector_docs):
    print(f"{i+1}. {format_docs([doc])}")

print("Final Answer:")
print(vector_answer)

                VECTOR-ONLY RAG SYSTEM OUTPUT

Retrieved Documents:
1. [Source: doc-2] Project 'Titan' is a new initiative focused on developing energy-efficient hardware. The primary goal is to reduce the power consumption of our data centers.
2. [Source: doc-4] Our company is committed to sustainability. Project 'Titan' is a core part of our green computing strategy.

Final Answer:
Based on the provided context, the company's power saving effort is an initiative called Project 'Titan', which is focused on developing energy-efficient hardware to reduce power consumption in data centers and is part of the green computing strategy. The context does not contain information about an error code for QLeap-V4 overheating.


### 4.2: Running the Keyword-Only RAG System

In [ ]:
keyword_docs = keyword_retriever.invoke(user_query)
keyword_answer = rag_chain_keyword.invoke(user_query)

print("="*60)
print("                KEYWORD-ONLY RAG SYSTEM OUTPUT")
print("="*60 + "\n")
print("Retrieved Documents:")
for i, doc in enumerate(keyword_docs):
    print(f"{i+1}. {format_docs([doc])}")

print("Final Answer:")
print(keyword_answer)

                KEYWORD-ONLY RAG SYSTEM OUTPUT

Retrieved Documents:
1. [Source: doc-3] A key feature of the QLeap-V4 is its advanced thermal management system. The official error code for overheating is 'ERR_THROTTLE_900'.
2. [Source: doc-2] Project 'Titan' is a new initiative focused on developing energy-efficient hardware. The primary goal is to reduce the power consumption of our data centers.

Final Answer:
Based on the context, the error code for QLeap-V4 overheating is 'ERR_THROTTLE_900'. Project 'Titan' is an initiative to reduce power consumption.


### 4.3: Running the Hybrid Search RAG System

In [ ]:
inputs = {"question": user_query}
hybrid_result = None
for output in hybrid_rag_app.stream(inputs, stream_mode="values"):
    hybrid_result = output

print("="*60)
print("                 HYBRID RAG SYSTEM OUTPUT")
print("="*60 + "\n")
print("Retrieved Documents:")
for i, doc in enumerate(hybrid_result['retrieved_docs']):
    print(f"{i+1}. {format_docs([doc])}")

print("Final Answer:")
print(hybrid_result['final_answer'])

--- [Hybrid Retriever] Running Vector and Keyword searches in parallel... ---
--- [Hybrid Retriever] Fused results: Found 3 unique documents. ---
--- [Generator] Synthesizing final answer from fused context... ---

                 HYBRID RAG SYSTEM OUTPUT

Retrieved Documents:
1. [Source: doc-2] Project 'Titan' is a new initiative focused on developing energy-efficient hardware. The primary goal is to reduce the power consumption of our data centers.
2. [Source: doc-4] Our company is committed to sustainability. Project 'Titan' is a core part of our green computing strategy.
3. [Source: doc-3] A key feature of the QLeap-V4 is its advanced thermal management system. The official error code for overheating is 'ERR_THROTTLE_900'.

Final Answer:
Our company's power saving effort is called Project 'Titan', which is a core part of our green computing strategy aimed at developing energy-efficient hardware to reduce data center power consumption. The official error code for QLeap-V4 overheati